In [153]:
# Dependencies and setup
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import matplotlib.pyplot as plt

from api_keys import census_key

In [154]:
# Import unemployment rate csv file, create DataFrame, and set index to Area Type
unemployment_data = Path('resources/Local_Area_Unemployment_Statistics__LAUS___Annual_Average.csv')
unemployment_df = pd.read_csv(unemployment_data)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
area_type_df = unemployment_df.loc[['State','County'],
                                   ['Area Name', 'Year', 'Labor Force', 'Employment', 'Unemployment','Unemployment Rate']]
area_type_df

# Group by Year and calculate averages
year_grouped = area_type_df.groupby(['Year', 'Area Name']).mean()
year_grouped

# Reduce to only 2017-2022
recent = year_grouped.loc[[2017, 2018, 2019, 2020, 2021, 2022]]
recent

Labor Force  Employment  Unemployment  \
Year Area Name                                                 
2017 Alameda County       838200.0    807100.0       31100.0   
     Alpine County           550.0       510.0          30.0   
     Amador County         14690.0     13960.0         740.0   
     Butte County         102000.0     96100.0        5900.0   
     Calaveras County      21020.0     20010.0        1000.0   
...                            ...         ...           ...   
2022 Tulare County        207500.0    190500.0       16900.0   
     Tuolumne County       19880.0     18990.0         890.0   
     Ventura County       413600.0    398400.0       15200.0   
     Yolo County          109000.0    104600.0        4400.0   
     Yuba County           31600.0     29900.0        1700.0   

                       Unemployment Rate  
Year Area Name                            
2017 Alameda County                  3.7  
     Alpine County                   6.2  
     Amador County                   5.0  
     Butte County                    5.8  
     Calaveras County                4.8  
...                                  ...  
2022 Tulare County                   8.2  
     Tuolumne County                 4.5  
     Ventura County                  3.7  
     Yolo County                     4.0  
     Yuba County                     5.5  

[354 rows x 4 columns]

In [155]:
# Import age csv file and create DataFrame
age_data = Path('resources/experiencing_homelessness_age_demographics.csv')
age_df = pd.read_csv(age_data)
age_df

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220
...,...,...,...,...,...
2136,2022,CA-614,San Luis Obispo County CoC,45-54,389
2137,2022,CA-614,San Luis Obispo County CoC,55-64,360
2138,2022,CA-614,San Luis Obispo County CoC,65+,174
2139,2022,CA-614,San Luis Obispo County CoC,Under 18,742


In [156]:
# Create clean DataFrame by dropping rows with '*' in count column
clean_age_df = age_df.loc[age_df['EXPERIENCING_HOMELESSNESS_CNT']!='*']
clean_age_df

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220
...,...,...,...,...,...
2136,2022,CA-614,San Luis Obispo County CoC,45-54,389
2137,2022,CA-614,San Luis Obispo County CoC,55-64,360
2138,2022,CA-614,San Luis Obispo County CoC,65+,174
2139,2022,CA-614,San Luis Obispo County CoC,Under 18,742


In [157]:
# Change homeless count data type to integer
clean_age_df['EXPERIENCING_HOMELESSNESS_CNT'] = clean_age_df["EXPERIENCING_HOMELESSNESS_CNT"].astype(int)

# Group by year and COC Name and sum homeless counts
total_homeless_count = clean_age_df.groupby(['CALENDAR_YEAR', 'COC_NAME'])['EXPERIENCING_HOMELESSNESS_CNT'].sum().reset_index()
total_homeless_count

C:\Users\mattcalvert\AppData\Local\Temp\ipykernel_12848\2318218614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_age_df['EXPERIENCING_HOMELESSNESS_CNT'] = clean_age_df["EXPERIENCING_HOMELESSNESS_CNT"].astype(int)


,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
263,2022,Stanislaus County CoC,6482
264,2022,Tehama County CoC,670
265,2022,Ventura County CoC,2919
266,2022,Yolo County CoC,1419


In [158]:
# Find all Continuums of Care in Los Angeles County
la_county_coc = total_homeless_count.loc[total_homeless_count["COC_NAME"].str.contains("Los Angeles County", case = False)]
la_county_coc

,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
9,2017,Glendale CoC (Los Angeles County),341
15,2017,Long Beach CoC (Los Angeles County),2754
16,2017,Los Angeles County CoC,57002
24,2017,Pasadena CoC (Los Angeles County),1328
53,2018,Glendale CoC (Los Angeles County),362
59,2018,Long Beach CoC (Los Angeles County),2746
60,2018,Los Angeles County CoC,76037
68,2018,Pasadena CoC (Los Angeles County),1352
97,2019,Glendale CoC (Los Angeles County),431
103,2019,Long Beach CoC (Los Angeles County),3265


In [159]:
# Use replace to group all of the Los Angeles County CoCs
cocs_to_replace = ["Glendale CoC (Los Angeles County)",
                   "Pasadena CoC (Los Angeles County)", 
                   "Long Beach CoC (Los Angeles County)"
                  ]
total_homeless_count['COC_NAME'] = total_homeless_count['COC_NAME'].replace(cocs_to_replace, "Los Angeles County CoC")
total_homeless_count

,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
263,2022,Stanislaus County CoC,6482
264,2022,Tehama County CoC,670
265,2022,Ventura County CoC,2919
266,2022,Yolo County CoC,1419


In [161]:
# Use Groupby again to group new Los Angeles County CoC rows
total_homeless_count = total_homeless_count.groupby(['CALENDAR_YEAR','COC_NAME'])['EXPERIENCING_HOMELESSNESS_CNT'].sum().reset_index()
total_homeless_count

,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
245,2022,Stanislaus County CoC,6482
246,2022,Tehama County CoC,670
247,2022,Ventura County CoC,2919
248,2022,Yolo County CoC,1419


In [94]:
# Import ethnicity csv file and create DataFrame
ethnicity_data = Path('resources/experiencing_homelessness_ethnicity_demographics.csv')
ethnicity_df = pd.read_csv(ethnicity_data)

In [95]:
# Import gender csv file and create DataFrame
gender_data = Path('resources/experiencing_homelessness_gender_demographics.csv')
gender_df = pd.read_csv(gender_data)

In [96]:
# Import race csv file and create DataFrame
race_data = Path('resources/experiencing_homelessness_race_demographics.csv')
race_df = pd.read_csv(race_data)